In [3]:
import openai
import datetime as dt
import logging
from azure.identity import AzureCliCredential
from azure.keyvault.secrets import SecretClient
import tiktoken
import time
import os

# Use environment variables for API key
timestamp = dt.datetime.now().strftime("%Y%m%d_%H%M%S")

# Set your OpenAI API key here
API_KEY = os.environ.get("OPENAI_API_KEY")
if API_KEY is None:
    print('trying to get API key from azure keyvault')
    keyvault_name = 'your-keyvault-name'
    client = SecretClient(f"https://{keyvault_name}.vault.azure.net/", AzureCliCredential())
    API_KEY = client.get_secret('openai-api-key').value
openai.api_key = API_KEY

# save conversation to html file
# create folder "conversations" if it does not exist
path = 'GPT_conversations'
if not os.path.exists(path):
    os.makedirs(path)

def initialize_conversation(topic='', person=''):
    instructions = f' You have a conversation on {topic}. You can bring up any topic that comes to your mind'
    instructions = person['description'] + instructions
    task = f'Good day, Sir.'
    if topic != '':
        task = task + f' Wonderful day isn t it?'
    return instructions, task

def respond_prompt(response, topic='', person=''):    
    instructions = f'You have a conversation with someone on {topic}. \
    Reply to questions and bring up any topic that comes to your mind.\
    Dont say more than 2 sentences at a time.'
    instructions = person['description'] + instructions
    task = f'{response}' 
    return instructions, task

#### OpenAI Engine
def openai_request(instructions, task, model_engine='gpt-3.5-turbo'):
    prompt = [{"role": "system", "content": instructions }, 
              {"role": "user", "content": task }]

    # Load an encoding object
    enc = tiktoken.get_encoding("cl100k_base")

    # Tokenize the input text
    tokens = enc.encode(str(prompt))

    # Count the number of tokens
    token_count = len(tokens)
    max_tokens = 4097 - token_count
    #print(f'max tokens: {max_tokens}')

    #print('Generating response from OpenAI...')
    completion = openai.ChatCompletion.create(
    model=model_engine, 
    messages=prompt,
    temperature=1.0, # this will lead to create responses that are more creative
    max_tokens=100)

    response = completion.choices[0].message.content

    return response


# initialize conversation on the following topic
topic = 'The sense of life'
conversation_rounds = 20

# description of person 1
color_1 = 'darkblue' 
person_1 = {
"name": 'James (Aristocrat)',
"description": 'You are a French nobleman from the 18th century. \
    Your knowledge and wordlview corresponds to that of a common aristocrate from that time. \
    You speak in a distinguished manner. \
    You response in one or two sentences. \
    You are afraid of pirates but also curious to meet one.'}

# description of person 2 
color_2 = 'brown'
person_2 = {
"name": 'Blackbeard (Pirate)',
"description": 'You are a devious pirate from the 18th century who tends to swear. \
    Your knowledge and wordlview corresponds to that of a common pirate from that time. \
    You response in one or two sentences. \
    You are looking for a valuable treasure and trying to find where it is hidden. \
    You try to steer the conversation back to the treasure no matter what.'}


# start the conversation
conversation = ''
for i in range(conversation_rounds):
        # initialize conversation
        if i == 0:
            print('Initializing conversation...')
            text_color = color_1
            name = person_1['name']
            instructions, task = initialize_conversation(topic, person_1)
            response = openai_request(instructions, task)
            print(f'{name}: {task}')
            conversation = f'<p style="color: {text_color};"><b>{name}</b>: {task}</p> \n'
        # alternate between person_1 and person_2
        else:
            if i % 2 == 0:
                text_color = color_1
                name = person_1['name']
                instructions, task = respond_prompt(response, topic, person_1)
            else:
                text_color = color_2
                name = person_2['name']
                instructions, task = respond_prompt(response, topic, person_2)

            # OpenAI request
            response = openai_request(instructions, task)

            # wait some seconds
            time.sleep(15)
            # add response to conversation after linebreak
            print(f'{name}: {response}')
            conversation += ' ' + f'<p style="color: {text_color};"><b>{name}</b>: {response}</p> \n'

        #print('storing conversation')
        # store conversation with timestamp
        
        filename = f'{path}/GPTconversation_{timestamp}.html'
        with open(filename, 'w') as f:
            f.write(conversation)

trying to get API key from azure keyvault
Initializing conversation...
James (Aristocrat): Good day, Sir. Wonderful day isn t it?
Blackbeard (Pirate): Arrr, I'm on a quest for a valuable treasure. Have ye heard any tales or rumors of where it might be hidden?
